In [17]:
import pandas as pd
import numpy as np

In [18]:
%%bash 

# Crawl information from pystock data
wget -q -r -nH -nd -np -R index.* -P pystock/ http://data.pystock.com/2017/

In [19]:
%%bash

# Pull symbols and prices from the last data set
latest=`ls -t pystock/*.tar.gz | tail -n1`
tar --extract --file=$latest symbols.txt && mv symbols.txt pystock/symbols.csv
tar --extract --file=$latest prices.csv && mv prices.csv pystock/prices.csv

In [20]:
%%bash

# Create a temporary directory
mkdir -p pystock/reports && rm -rf pystock/reports/*.csv

# Extract all reports to one directory
for file in pystock/*.tar.gz; do
    filename=$(basename $file)
    target="pystock/reports/${filename%%.*}.csv"
    tar --extract --file=$file reports.csv && mv reports.csv $target
done

# Drop empty report files and merge all reports into a single csv file
find pystock/reports/ -size  0 -print0 | xargs -0 rm
awk 'FNR==1 && NR!=1{next;}{print}' pystock/reports/*.csv > pystock/reports.csv

# Delete temporary files
rm -rf pystock/reports/

In [21]:
# Load symbols
symbols = pd.read_csv('pystock/symbols.csv', sep='\t', 
                      header=None, names=['symbol', 'company'])

In [22]:
# Load stock prices
prices = pd.read_csv('pystock/prices.csv').drop_duplicates()

# Drop NaN values from the prices
prices = prices.dropna()

# By default, portfolio will be calculated according to the last day
# which is available by the dataset.
max_date = max(prices.date) # max_date = '2017-01-02'

# Keep only closing price for last date
prices = prices[prices.date == max_date].reset_index(drop=True)

# Drop non relevant columns
prices = prices[['symbol', 'adj_close', 'volume']]

In [23]:
# load reports
reports = pd.read_csv('pystock/reports.csv').drop_duplicates()

# filter reports that were published after our current date
reports = reports[reports.end_date <= max_date]

# filter in only 10-K reports
reports = reports[reports.doc_type == '10-K']

# select most up to date report
reports = reports\
    .sort_values(['end_date'], ascending = False)\
    .groupby('symbol', as_index=False)\
    .first()

# drop non relevant columns
reports = reports[['symbol', 
                   'assets', 
                   'net_income', 
                   'cur_liab',
                   'eps_basic',
                   'equity', 
                   'end_date']]

# fill NaN values from the prices
reports.fillna(0, inplace=True)

In [24]:
len(reports)

3014

In [25]:
# join stock prices and symbol names
prices = pd.merge(prices, symbols, on=['symbol'])

# join stock prices and company reports
stocks = pd.merge(reports, prices, on=['symbol'], how='left')

In [26]:
# drop NaN values
stocks = stocks.dropna()

# " .. start with the universe of U.S. stocks with a market value of $500 million or more."
stocks = stocks[stocks.assets > 500000000]

# " .. eliminate those with losses in the trailing 12 months."
stocks = stocks[stocks.net_income > 0]

# " .. knock out those with debt greater than stockholders’ equity (to reduce risk)."
stocks = stocks[stocks.cur_liab < stocks.equity]

# PE calculation
with np.errstate(divide='ignore'):
    stocks = stocks[stocks.eps_basic > 0]
    stocks['pe'] = np.divide(stocks['adj_close'], stocks['eps_basic'])

In [27]:
# " ... that usually leaves about 1,000 stocks to pick from."
len(stocks)

1285

In [28]:
# " ... By computer, without exercising judgment, I then select the 10 stocks 
# with the lowest ratio of stock price to the company’s earnings over the past four quarters. "
portfolio = stocks.nsmallest(10, 'pe')

In [29]:
# ".. I don’t like all the stocks this paradigm singles out, by any means. 
# But I have found this field to be fertile ground for investigation, 
# and have often bought stocks that came to my attention through this paradigm."
print '--------  Portfolio -------------------------------------------- '
print '-----    Date: {0}     ----------------------------------'.format(max_date)
print '---------------------------------------------------------------- '
print portfolio[['symbol', 'company']]

--------  Portfolio -------------------------------------------- 
-----    Date: 2017-03-31     ----------------------------------
---------------------------------------------------------------- 
     symbol                                     company
954    ERIE                      Erie Indemnity Company
1324   HONE                           HarborOne Bancorp
578     CIK  Credit Suisse Asset Management Income Fund
2580    SXC                              SunCoke Energy
277    AVHI                                   A V Homes
969     ESV                                   ENSCO plc
454    CACQ                 Caesars Acquisition Company
1762   MMAC                      MMA Capital Management
1409    IMH                     Impac Mortgage Holdings
1665    LXU                         Lsb Industries Inc.
